# FESDModel

FESD - Fault estimation for skeleton detection - is a suite that aims at finding faults in joints of skeletons, which are detected by human pose estimatiors.

FESDData is the sister project to this notebook, which aims at recording depth and rgb data, as well as populating the data with human poses from variing human pose estimators.

Furthermore, FESTData augments all data based on joint confidence.

FFESDModel aims to develop and evaluate a model based on the faulty and augmented joint data as well as RGBD data.

## Libraries

We need a range of libraries which are imported here. We also define some constants.

In [30]:
import os
import json
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tqdm

import cv2
import pyrealsense2 as rs

sns.set_style()

In [31]:
RECORDING_DIR = Path('H:/Recordings/')

## Data Loading

Firstly we need to import all the recordings into the notebook.


### Load Metadata

It is important to load the metadata, such as the session parameters, the exercises and the recording paths.

In [32]:
recording_jsons = []
for file in os.listdir(RECORDING_DIR):
  if (file.endswith('.json')):
    with open(file=os.path.join(RECORDING_DIR, file), mode='r') as file:
      data = json.load(file)
      recording_jsons.append(data)

len(recording_jsons)

1

Then we load the Exercises from the exercise file.

In [33]:
with open(file="Exercises.json", mode='r') as file:
  exercises_json = json.load(file)['Exercises']

with open(file="JointErrors.json", mode='r') as file:
  joint_error_json = json.load(file)

with open(file="SkeletonErrors.json", mode='r') as file:
  skeleton_error_json = json.load(file)

len(exercises_json)

13

## Load Frame

Here we define the load frame functions. For now we focus on nuitrack recordings.

In [34]:
def id_2_name(i: int):
  return 'frame_' + str(i) + '.yml'

In [42]:
def load_frame(session: json, frame_id: int) -> (np.ndarray, json):
  frame_path = RECORDING_DIR /  session['Cameras'][0]['FileName'] / id_2_name(frame_id)
  frame_file = cv2.FileStorage(str(frame_path), cv2.FileStorage_READ)
  frame = frame_file.getNode('frame').mat()
  with open(file=RECORDING_DIR /  session['Skeleton'], mode='r') as file:
    skeleton_json = json.load(file)
  return np.asarray( frame[:,:] ), skeleton_json

In [43]:
frame, skeleton = load_frame(recording_jsons[0], 0)
frame.shape

(480, 640, 4)

In [37]:
origin_joint_index_op = 0
origin_joint_index_nui = 3

def load_skeletons(skeletons_json, origin_joint_index):
  skeleton_frames = []
  for frame in skeletons_json:
    joints = []
    person_count = 0
    for person in frame:
      if person['valid']:
        joints = []
        person_count += 1
        for joint in person['Skeleton']:
          if joint['valid']:
            joints.append([
              joint['x'] - person['Skeleton'][origin_joint_index]['x'],
              joint['y'] - person['Skeleton'][origin_joint_index]['y'],
              joint['z'] - person['Skeleton'][origin_joint_index]['z'],
              joint['score']
            ])
          else:
            joints.append(None)
    if person_count > 1:
      print("Warning: More than one person in frame, only using last person!")
    skeleton_frames.append(joints)
  return skeleton_frames